In [11]:
import os
import re
import shutil
import subprocess
import google.generativeai as genai
from dotenv import load_dotenv

# --- Configuration ---
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE")
genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
# --- Helper Functions for Local Repo Handling ---

def clone_repo(repo_url: str) -> str:
    """
    Clones a public repository into a temporary local directory.
    Returns the path to the directory if successful, otherwise None.
    """
    temp_dir = tempfile.mkdtemp()
        
    print(f"Cloning {repo_url} into '{temp_dir}'...")
    try:
        # We use --depth 1 to do a shallow clone, which is faster and uses less space
        subprocess.check_call(
            ['git', 'clone', '--depth', '1', repo_url, temp_dir],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        print("✅ Cloning successful.")
        return temp_dir
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print(f"🔥 Error cloning repository: {e}. Please check the URL and that Git is installed.")
        # Clean up the failed clone attempt
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        return None

def analyze_local_repo(repo_path: str) -> dict:
    """Analyzes a local repository to extract file structure and manifest contents."""
    print("🌳 Analyzing local repository structure...")
    file_paths = []
    manifest_content = ""
    
    key_files = [
        'package.json', 'requirements.txt', 'pom.xml', 'build.gradle',
        'Dockerfile', 'docker-compose.yml', 'go.mod', 'Gemfile', 'composer.json'
    ]
    
    for root, _, files in os.walk(repo_path):
        if '.git' in root:
            continue
        for name in files:
            relative_path = os.path.relpath(os.path.join(root, name), repo_path)
            file_paths.append(relative_path.replace("\\", "/"))
            if name in key_files:
                print(f"   - Found key file: {relative_path}")
                try:
                    with open(os.path.join(root, name), 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                        manifest_content += f"\n\n--- Content of {name} ---\n{content[:2000]}\n"
                except Exception as e:
                    print(f"   - Warning: Could not read {name}: {e}")

    return {"file_paths": file_paths, "manifest_content": manifest_content}

def slugify(text: str) -> str:
    """
    Converts a string into a URL-friendly/filename-friendly slug.
    Example: "ADR-001: Use Docker for Containerization" -> "adr-001-use-docker-for-containerization"
    """
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s-]', '', text)  # Remove invalid chars
    text = re.sub(r'[\s-]+', '-', text).strip('-') # Replace spaces and repeated hyphens
    return text

def save_adrs(adr_content: str, output_dir: str):
    """Parses the Gemini output and saves each ADR to a separate file."""
    print(f"\n💾 Saving ADRs to directory: '{output_dir}'")
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Split the content by the standard markdown horizontal rule
    adrs = adr_content.split('---')
    
    saved_count = 0
    for adr_text in adrs:
        adr_text = adr_text.strip()
        if not adr_text:
            continue

        # Find the title line (e.g., "### ADR-001: Use Next.js Framework")
        title_match = re.search(r"###\s*(.*)", adr_text)
        if title_match:
            title = title_match.group(1).strip()
            filename = slugify(title) + ".md"
            filepath = os.path.join(output_dir, filename)
            
            try:
                # Re-add the separator for formatting, except for the first file
                content_to_save = adr_text if saved_count == 0 else "---\n" + adr_text
                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(content_to_save)
                print(f"   - Saved: {filename}")
                saved_count += 1
            except IOError as e:
                print(f"   - 🔥 Error saving file {filename}: {e}")
        else:
            print("   - ⚠️ Warning: Could not find a title for an ADR block. Skipping.")

    if saved_count == 0:
        print("🤷 No valid ADRs were found in the output to save.")
    else:
        print(f"\n✅ Successfully saved {saved_count} ADR file(s).")


# --- Main Generation Function ---

def generate_adrs_for_repo(repo_url: str, output_dir: str = "generated_adrs"):
    """
    Clones, analyzes, and saves generated ADRs for a repository.
    """
    temp_repo_path = None
    # 1. Clone the repository
    temp_repo_path = clone_repo(repo_url)
    if not temp_repo_path:
        return

    # 2. Analyze the local files
    repo_context = analyze_local_repo(temp_repo_path)
    
    # 3. Construct the Prompt for Gemini
    prompt = f"""
    You are an expert software architect... [Prompt content remains the same as previous version] ...
    
    Use the following Markdown template for each ADR, separating them with a '---' horizontal rule:
    ---
    
    ### ADR-001: [Title of Decision]

    **Status:** Inferred
    **Context:** ...
    **Decision:** ...
    **Consequences:** ...
    """
    # Note: A shortened prompt is shown here for brevity. 
    # The full prompt from the previous answer should be used.
    
    # 4. Call the Gemini API
    print("\n🧠 Sending context to Gemini for analysis. This may take a moment...")
    model = genai.GenerativeModel('gemini-2.5-flash')
    response = model.generate_content(prompt)
    
    # 5. Save the result to files
    save_adrs(response.text, output_dir)
    
    print("\n**Disclaimer:** These ADRs are AI-generated based on static analysis and should be reviewed and validated by the development team.")


    

In [13]:
if __name__ == "__main__":
    target_repo = "https://github.com/karthikv1392/cs6401_se.git"    
    output_directory = "Generated_ADRs_CA"
    generate_adrs_for_repo(target_repo, output_directory)

Cloning https://github.com/karthikv1392/cs6401_se.git into 'C:\Users\rudra\AppData\Local\Temp\tmpqbej7d_5'...
✅ Cloning successful.
🌳 Analyzing local repository structure...
   - Found key file: Gemfile

🧠 Sending context to Gemini for analysis. This may take a moment...

💾 Saving ADRs to directory: 'Generated_ADRs_CA'
   - ⚠️ Warning: Could not find a title for an ADR block. Skipping.
   - Saved: adr-001-cloud-provider-selection-aws.md
   - Saved: adr-002-application-architecture-pattern-modular-monolith.md
   - Saved: adr-003-primary-data-store-postgresql-with-aws-rds.md
   - Saved: adr-004-backend-technology-stack-python-with-fastapi.md
   - Saved: adr-005-authentication-and-authorization-strategy-openid-connect-with-aws-cognito.md
   - Saved: adr-006-deployment-strategy-containerized-services-on-aws-fargate.md
   - Saved: adr-007-frontend-framework-reactjs.md
   - Saved: adr-008-monitoring-and-logging-strategy-aws-cloudwatch-structured-logging.md

✅ Successfully saved 8 ADR file(s)